In [1]:
import Pkg
using HTTP
using JSON
using Dates
using CSV

using Parameters
using Glob
using DataFrames

In [12]:
struct Asset
    symbol::Symbol
    Asset(s::String) = new(Symbol(s))
end

## IEX TRADING ##

struct IEXTradingAPI <: API
    base::String
end
IEXTradingAPI() = IEXTradingAPI("https://ws-api.iextrading.com/1.0")

function FetchSymbols(iex::IEXTradingAPI)
    endpoint = "/ref-data/symbols"
    JSON.Parse(Request(iex,endpoint))
end

## ALPHADVANTAGE ##

struct AlphadvantageAPI <: API
    key::String
    base::String
end
AlphadvantageAPI() = AlphadvantageAPI("3J1NP4X808DF2YY8", "https://www.alphavantage.co/query?")

Format(api::AlphadvantageAPI, endpoint::String) = api.base * endpoint * "&apikey=" * api.key

function FetchHistory(api::AlphadvantageAPI, asset::Asset, method::String, outputsize::String; datatype=nothing)
    endpoint = "symbol=$(asset.symbol)&"
    endpoint *= "function=$method&"
    endpoint *= "outputsize=$outputsize"
    
    !isnothing(datatype) ? endpoint *= "&datatype=$datatype" : nothing 
    
    Request(api, endpoint)
end

FetchWeeklyHistory(api::AlphadvantageAPI, asset::Asset, outputsize::String; datatype=nothing) =
    FetchHistory(api, asset, "TIME_SERIES_WEEKLY_ADJUSTED", outputsize, datatype=datatype)

FetchDailyHistory(api::AlphadvantageAPI, asset::Asset, outputsize::String; datatype=nothing) =
    FetchHistory(api, asset, "TIME_SERIES_DAILY_ADJUSTED", outputsize, datatype=datatype)

## ASSET ##

@with_kw mutable struct AssetHistory
    asset::Asset
    dates::Array{Date}                   = Array{Date,1}()
    open::Array{<:AbstractFloat}         = zeros()
    high::Array{<:AbstractFloat}         = zeros()
    close::Array{<:AbstractFloat}        = zeros()
    close_adjusted::Array{<:AbstractFloat} = zeros()
    low::Array{<:AbstractFloat}          = zeros()
    volume::Array{<:AbstractFloat}       = zeros()
    dividend::Array{<:AbstractFloat}     = zeros()
    split_coef::Array{<:AbstractFloat}   = zeros()
end
AssetHistory(symbol::String) = AssetHistory(Symbol(symbol))
AssetHistory(asset::Asset) = AssetHistory(asset.symbol)

function AssetHistory(asset::Asset, df::DataFrame)
    AssetHistory(
        asset, Date.(df.timestamp), Array{Float64,1}(df.open), Array{Float64,1}(df.high), 
        Array{Float64,1}(df.low), Array{Float64,1}(df.close), Array{Float64,1}(df.adjusted_close),
        Array{Float64,1}(df.volume), Array{Float64,1}(df.dividend_amount), 
        Array{Float64,1}(df.split_coefficient))
end
 

function AddFromDate(asset::AssetHistory, date::String, dict::Dict)
    date = int.(split(date,"-"))
    date = Date(date...)

    push!(asset.dates, date)
    push!(asset.open, _dict["1. open"])
    push!(asset.high, _dict["2. high"])
    push!(asset.low, _dict["3. low"])
    push!(asset.close, _dict["4. close"])
    push!(asset.close_adjusted, _dict["5. adjusted close"])
    push!(asset.volume, _dict["6. volume"])
    push!(asset.dividend, _dict["7. dividend amount"])
    push!(asset.split_coef, _dict["8. split coefficient"])
end

function AddFromHistory(asset::AssetHistory, history::Dict)
    for (date,value) in history
        AddFromDate(asset,date,value)
    end
end

function SaveStockHistory(asset::Asset; outputsize="full", API::API = AlphadvantageAPI())

    r = FetchDailyHistory(API, asset, "full", datatype="csv")

    open("resources/$(asset.symbol).csv", "w+") do io
       write(io, r) 
    end 
end

function SaveStocksHistory()
    df = CSV.File("resources/1billion_companies.tsv", delim='\t') 
    aa = AlphadvantageAPI()
    i = 0
    for row in df
        symbol = row.Symbol
        if symbol === missing
            continue
        end    

        if row.Sector == "n/a"
            continue
        end

        asset = Asset(symbol)
        SaveStockHistory(asset, outputsize="full", API=aa)

        i += 1
        if i > 100
            break
        end
        sleep(10)
    end
end

abstract type API end

Format(api::API, endpoint::String) = api.base * endpoint
    
function Request(api::API, endpoint::String)
    url = Format(api, endpoint)
    println("Requesting $url")
    r = HTTP.request("GET", url)
    
    r.body
end



## Investment & Portfolio ##

abstract type InvestmentType end
abstract type LongInvestment <: InvestmentType end
abstract type ShortInvestment <: InvestmentType end 

struct InvestmentReturn
    value::AbstractFloat
    percentage::AbstractFloat
end

abstract type AbstractInvestment end

struct Investment{T} <: AbstractInvestment where T <: InvestmentType
    asset::Asset
    value::AbstractFloat
    dateOpen::Date
end

struct ClosedInvestment{T} <: AbstractInvestment where T <: InvestmentType
    asset::Asset
    valueOpen::AbstractFloat
    valueClose::AbstractFloat
    dateOpen::DateTime
    dateClosed::DateTime
    closed::InvestmentReturn
end


abstract type AbstractPortfolio end

struct Portfolio <: AbstractPortfolio 
    Investments::Array{<:AbstractInvestment}
end
Add!(portfolio::AbstractPortfolio, inv::AbstractInvestment) = push!(portfolio.Investments, inv)

function Long!(portfolio::AbstractPortfolio, asset::Asset, value::AbstractFloat; 
        dateOpen::Date = Dates.today(), dateClose::Union{Date,Nothing} = nothing)
    
    inv = Investment{LongInvestment}(asset, value, dateOpen, dateClose)
    add!(portfolio, inv)
end

function Short!(portfolio::AbstractPortfolio, asset::Asset, value::AbstractFloat; 
        dateOpen::Date = Dates.today(), dateClose::Union{Date,Nothing} = nothing)
    
    inv = Investment{ShortInvestment}(asset, value, dateOpen, dateClose)
    add!(portfolio, inv)
end

function Close!(pf::AbstractPortfolio, inv::Investment)
    findfirst(pf.Investments, x => x == inv)
end


Return(inv::Investment{LongInvestment}, value::AbstractFloat) = value - inv.value
Return(inv::Investment{ShortInvestment}, value::AbstractFloat) = inv.value - value

PotentialProfit(inv::AbstractInvestment) = 0.
ClosedProfit(inv::AbstractInvestment) = 0.
ClosedPercentage(inv::AbstractInvestment) = 0.

ClosedProfit(inv::ClosedInvestment) = inv.close.value
ClosedPercentage(inv::ClosedInvestment) = inv.close.percentage

function PotentialProfit(inv::Investment; currentValue::Union{AbstractFloat,Nothing} = nothing,
    dateTime::DateTime = Dates.now())
    
    if isnothing(currentValue)
        currentValue = FetchAssetValue(inv.asset, dateTime)
    end
    
    return Return(inv, currentValue) 
end

function PotentialProfit(pf::AbstractPortfolio)
    total = 0.
    for inv in pf.investestements
        total += PotentialProfit(inv)
    end
end

function ClosedProfit(pf::AbstractPortfolio)
    total = 0.
    for inv in pf.investestements
        total += ClosedProfit(inv)
    end
end

ClosedProfit (generic function with 3 methods)

In [7]:
# SaveStocksHistory()

Requesting https://www.alphavantage.co/query?symbol=MSFT&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=AMZN&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=AAPL&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=GOOGL&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=GOOG&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=FB&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=CSCO&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&api

Requesting https://www.alphavantage.co/query?symbol=PAYX&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=TEAM&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=MELI&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=ORLY&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=EA&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=ALXN&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apikey=3J1NP4X808DF2YY8
Requesting https://www.alphavantage.co/query?symbol=GOLD&function=TIME_SERIES_DAILY_ADJUSTED&outputsize=full&datatype=csv&apik

In [20]:
assetHistories = Dict()
for path in glob("*.csv", "resources/Top100Companies/")
    symbol = split( split(path, '/')[end], '.')[1]
    asset = Asset(String(symbol))
    
    @show "Loading $(symbol)"
    
    df = CSV.read(path)
    hist = AssetHistory(asset, df)
    assetHistories[symbol] = hist
end

"Loading $(symbol)" = "Loading AABA"
"Loading $(symbol)" = "Loading AAPL"
"Loading $(symbol)" = "Loading ADBE"
"Loading $(symbol)" = "Loading ADI"
"Loading $(symbol)" = "Loading ADP"
"Loading $(symbol)" = "Loading ADSK"
"Loading $(symbol)" = "Loading ALGN"
"Loading $(symbol)" = "Loading ALXN"
"Loading $(symbol)" = "Loading AMAT"
"Loading $(symbol)" = "Loading AMD"
"Loading $(symbol)" = "Loading AMGN"
"Loading $(symbol)" = "Loading AMOV"
"Loading $(symbol)" = "Loading AMTD"
"Loading $(symbol)" = "Loading AMZN"
"Loading $(symbol)" = "Loading ATVI"
"Loading $(symbol)" = "Loading AVGO"
"Loading $(symbol)" = "Loading BIDU"
"Loading $(symbol)" = "Loading BIIB"
"Loading $(symbol)" = "Loading BKNG"
"Loading $(symbol)" = "Loading CDNS"
"Loading $(symbol)" = "Loading CELG"
"Loading $(symbol)" = "Loading CERN"
"Loading $(symbol)" = "Loading CHTR"
"Loading $(symbol)" = "Loading CMCSA"
"Loading $(symbol)" = "Loading CME"
"Loading $(symbol)" = "Loading COST"
"Loading $(symbol)" = "Loading CSCO"
"Loa

In [23]:
date = Date("2018-09-01")

for (symbol, hist) in assetHistories
    idx = findfirst(assetHistories, x->x.Date == date)
    
end



1

In [27]:
df = CSV.read("resources/Top100Companies/MSFT.csv")

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount
,Date,Float64,Float64,Float64,Float64,Float64,Int64,Float64
1,2019-06-28,134.57,134.6,133.156,133.96,133.96,30017029,0.0
2,2019-06-27,134.14,134.71,133.51,134.15,134.15,16557482,0.0
3,2019-06-26,134.35,135.74,133.6,133.93,133.93,23657745,0.0
4,2019-06-25,137.25,137.59,132.73,133.43,133.43,33327420,0.0
5,2019-06-24,137.0,138.4,137.0,137.78,137.78,20628841,0.0
6,2019-06-21,136.58,137.73,136.46,136.97,136.97,36727892,0.0
7,2019-06-20,137.45,137.66,135.72,136.95,136.95,33042592,0.0
8,2019-06-19,135.0,135.93,133.81,135.69,135.69,23744441,0.0
9,2019-06-18,134.19,135.24,133.57,135.16,135.16,25934458,0.0
